In [8]:
#PART1
#You will need to manually amend the target datasets for this script

library(bigrquery)
library(pryr)

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

# Amend these to "point" at the datasets you want to query
targetdb1 <-'yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8'
targetdb1 <-gsub(' ','',targetdb1)
print (targetdb1)

targetdb2 <-'yhcr-prd-phm-bia-core.CB_MYSPACE_JDB'
targetdb2 <-gsub(' ','',targetdb2)
print (targetdb2)

[1] "yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8"
[1] "yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"


In [9]:
#Were going to build a temporary cohort table in our personal space

sql3 <-paste('create or replace table ',targetdb2,'.tmp_cohort 
as SELECT distinct b.person_id,
DATE_DIFF(CURRENT_DATE(),b.birth_datetime,YEAR) as age
FROM ',targetdb1,'.person b 
where DATE_DIFF(CURRENT_DATE(),birth_datetime,YEAR) >= 64 
and DATE_DIFF(CURRENT_DATE(),birth_datetime,YEAR) <=95
and death_datetime is not null ;', sep = "")
#This runs it
tb3 <- bq_project_query(project_id, sql3)

#We don't need to load it into a dataframe
#table <- bq_table_download(tb3)


In [11]:
# Now we join our cohort table : tmp_cohort
# to the sr_code table 
#and then run it.

sql5 <-paste('select distinct 
b.person_id, 
a.ctv3code
 from ',targetdb1,'.tbl_srcode a,
',targetdb2,'.tmp_cohort b 
where a.person_id = b.person_id 
limit 1000000', sep = "")
#This runs it
tb5 <- bq_project_query(project_id, sql5)

#This loads  it into an R data frame
table2 <- bq_table_download(tb5)
#This displays it
#table2

object_size(table2)

13.76 MB